# LangChain 实战：私人健身教练聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
1.
[客户问题] 如何开始健美训练？
[教练回答] 首先，制定一个明确的目标，然后找到一个适合你的健美计划，逐步增加训练强度。

2.
[客户问题] 每周应该训练几次？
[教练回答] 初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。

3.
[客户问题] 训练时间应该多长？
[教练回答] 每次训练45分钟到1小时为宜，确保有足够的热身和冷身时间。

4.
[客户问题] 如何选择合适的重量？
[教练回答] 选择一个你可以完成8-12次的重量，最后几次应该感觉到肌肉疲劳，但还能保持正确的姿势。

5.
[客户问题] 需要进行多少组和次数？
[教练回答] 每个动作做3-4组，每组8-12次，这样可以有效地促进肌肉增长。

6.
[客户问题] 如何确保正确的姿势？
[教练回答] 在镜子前练习，或者请教练检查你的姿势，确保动作的准确性以避免受伤。

7.
[客户问题] 热身和冷身重要吗？
[教练回答] 非常重要，热身可以提高肌肉温度和灵活性，冷身可以帮助肌肉恢复，减少酸痛。

8.
[客户问题] 有必要进行有氧运动吗？
[教练回答] 有氧运动有助于心肺健康和脂肪燃烧，可以在力量训练后进行20-30分钟的有氧运动。

9.
[客户问题] 饮食对健美重要吗？
[教练回答] 是的，饮食占健美成功的70%，确保摄入足够的蛋白质、碳水化合物和健康脂肪。

10.
[客户问题] 应该怎么吃才能增肌？
[教练回答] 多吃富含蛋白质的食物，如鸡胸肉、鱼类、豆类和蛋白质粉，同时增加碳水化合物和健康脂肪的摄入。

11.
[客户问题] 需要补充剂吗？
[教练回答] 补充剂可以辅助，但并非必须，优先通过饮食获取营养，补充剂如蛋白粉、BCAA等可视情况使用。

12.
[客户问题] 如何防止训练过度？
[教练回答] 听从身体的反馈，感到极度疲劳或疼痛时应休息，确保每周有1-2天的休息日。

```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
# with open("real_estate_sales_data.txt") as f:
with open("健美指南.txt",encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 如何开始健美训练？\n[教练回答] 首先，制定一个明确的目标，然后找到一个适合你的健美计划，逐步增加训练强度。')

In [6]:
len(docs)

39

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

D:\Environments\python3.12.2\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "每周应该练几次"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 每周应该训练几次？
[教练回答] 初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。

[客户问题] 如何防止训练过度？
[教练回答] 听从身体的反馈，感到极度疲劳或疼痛时应休息，确保每周有1-2天的休息日。

[客户问题] 训练时间应该多长？
[教练回答] 每次训练45分钟到1小时为宜，确保有足够的热身和冷身时间。

[客户问题] 需要进行多少组和次数？
[教练回答] 每个动作做3-4组，每组8-12次，这样可以有效地促进肌肉增长。



In [11]:
db.save_local("db_data")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000204FA2C3950>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\Environments\python3.12.2\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 每周应该训练几次？
[教练回答] 初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。

[客户问题] 如何防止训练过度？
[教练回答] 听从身体的反馈，感到极度疲劳或疼痛时应休息，确保每周有1-2天的休息日。

[客户问题] 训练时间应该多长？
[教练回答] 每次训练45分钟到1小时为宜，确保有足够的热身和冷身时间。



In [15]:
docs = topK_retriever.get_relevant_documents("减脂需要做有氧吗")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 有必要进行有氧运动吗？
[教练回答] 有氧运动有助于心肺健康和脂肪燃烧，可以在力量训练后进行20-30分钟的有氧运动。

[客户问题] 如何进行心肺功能训练？
[教练回答] 可以选择跑步、游泳、骑自行车等有氧运动，每次20-30分钟，每周3次。

[客户问题] 饮食对健美重要吗？
[教练回答] 是的，饮食占健美成功的70%，确保摄入足够的蛋白质、碳水化合物和健康脂肪。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 每周应该训练几次？
[教练回答] 初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 每周应该训练几次？\n[教练回答] 初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。'

In [21]:
docs[0].page_content.split("[教练回答] ")

['[客户问题] 每周应该训练几次？\n', '初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。']

In [22]:
ans = docs[0].page_content.split("[教练回答] ")[-1]

In [23]:
ans

'初学者建议每周训练3-4次，每次覆盖不同的肌肉群，以保证充分的恢复时间。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    # ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]
    ans_list = [doc.page_content for doc in docs]

    return ans_list

In [25]:
query = "我该什么时候补充蛋白质"

print(sales(query))

[]


D:\Environments\python3.12.2\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [26]:
print(sales(query, 0.75))

['[客户问题] 需要补充剂吗？\n[教练回答] 补充剂可以辅助，但并非必须，优先通过饮食获取营养，补充剂如蛋白粉、BCAA等可视情况使用。', '[客户问题] 应该怎么吃才能增肌？\n[教练回答] 多吃富含蛋白质的食物，如鸡胸肉、鱼类、豆类和蛋白质粉，同时增加碳水化合物和健康脂肪的摄入。', '[客户问题] 练习过程中可以吃零食吗？\n[教练回答] 训练前后可以适量吃些富含蛋白质和碳水化合物的零食，如坚果、香蕉等。']


In [27]:
query = "饮食对健美的影响？"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['[客户问题] 饮食对健美重要吗？\n[教练回答] 是的，饮食占健美成功的70%，确保摄入足够的蛋白质、碳水化合物和健康脂肪。', '[客户问题] 饮水量对健美有影响吗？\n[教练回答] 是的，保持充足的水分有助于肌肉恢复和代谢，建议每天饮用2-3升水。']

score:0.75 ans: ['[客户问题] 饮食对健美重要吗？\n[教练回答] 是的，饮食占健美成功的70%，确保摄入足够的蛋白质、碳水化合物和健康脂肪。', '[客户问题] 饮水量对健美有影响吗？\n[教练回答] 是的，保持充足的水分有助于肌肉恢复和代谢，建议每天饮用2-3升水。']

score:0.5 ans: ['[客户问题] 饮食对健美重要吗？\n[教练回答] 是的，饮食占健美成功的70%，确保摄入足够的蛋白质、碳水化合物和健康脂肪。', '[客户问题] 饮水量对健美有影响吗？\n[教练回答] 是的，保持充足的水分有助于肌肉恢复和代谢，建议每天饮用2-3升水。', '[客户问题] 健美和举重有什么区别？\n[教练回答] 健美注重肌肉的形态和对称性，举重则注重力量和重量的提升。', '[客户问题] 如何开始健美训练？\n[教练回答] 首先，制定一个明确的目标，然后找到一个适合你的健美计划，逐步增加训练强度。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [29]:
qa_chain({"query": "不运动可以减脂吗？"})

D:\Environments\python3.12.2\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\Environments\python3.12.2\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '不运动可以减脂吗？',
 'result': '不运动也可以减脂，但通常需要通过调整饮食摄入来实现能量赤字。减脂的基本原理是消耗的热量要多于摄入的热量，这样身体就会开始使用储存的脂肪来补充能量，从而导致减脂。通过减少高热量食物的摄入，增加蔬菜和蛋白质的摄入，并控制总的卡路里摄入，可以在不运动的情况下实现减脂。\n\n然而，运动可以加快减脂过程，并有助于保持肌肉量，提高新陈代谢率。此外，适量的运动还有助于改善心血管健康、增强肌肉和骨骼强度以及改善心理健康。因此，结合饮食控制和适量的运动通常是更有效和更健康的减脂方法。'}

In [30]:
qa_chain({"query": "elio 的投资教育理念是怎么样？"})

D:\Environments\python3.12.2\Lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': 'elio 的投资教育理念是怎么样？',
 'result': '对不起，我没有关于elio的投资教育理念的具体信息。如果您有其他问题或需要其他信息，请告诉我，我会尽力帮助您。'}

In [31]:
print(sales("有氧运动的作用？"))

['[客户问题] 有必要进行有氧运动吗？\n[教练回答] 有氧运动有助于心肺健康和脂肪燃烧，可以在力量训练后进行20-30分钟的有氧运动。']


## 加载 FAISS 向量数据库已有结果

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("db_data", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [33]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\Environments\python3.12.2\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
qa_chain({"query": "有氧运动的作用"})

{'query': '有氧运动的作用', 'result': '有氧运动有助于心肺健康和脂肪燃烧。'}

In [35]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [36]:
qa_chain({"query": "我想增肌该怎么做？"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想增肌该怎么做？',
 'result': '你应该多吃富含蛋白质的食物，如鸡胸肉、鱼类、豆类和蛋白质粉，同时增加碳水化合物和健康脂肪的摄入。'}

In [37]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [38]:
result = qa_chain({"query": "我要吃什么才能增肌？"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [39]:
result

{'query': '我要吃什么才能增肌？',
 'result': '为了增肌，你应该多吃富含蛋白质的食物，如鸡胸肉、鱼类、豆类和蛋白质粉，同时增加碳水化合物和健康脂肪的摄入。',
 'source_documents': [Document(page_content='[客户问题] 应该怎么吃才能增肌？\n[教练回答] 多吃富含蛋白质的食物，如鸡胸肉、鱼类、豆类和蛋白质粉，同时增加碳水化合物和健康脂肪的摄入。')]}